In [ ]:
import pickle
import gzip
import pandas as pd

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/del_X_train.pickle','rb') as f:
    del_X_train = pickle.load(f)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/bigtrain_y.pickle','rb') as f:
    del_y_train = pickle.load(f)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/pad_X_test.pickle','rb') as f:
    pad_X_test = pickle.load(f)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/pad_X_train.pickle','rb') as f:
    pad_X_train = pickle.load(f)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/token_X_test.pickle','rb') as f:
    token_X_test = pickle.load(f)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/pad_X_val.pickle','rb') as f:
    pad_X_val = pickle.load(f)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/bigval_y.pickle','rb') as f:
    bigval_y = pickle.load(f)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/Pre3.csv')

**모델 4**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
import os
tf.random.set_seed(777)

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks"
vocab_size= 18658
max_len = 300
val_rate = 0.2

In [ ]:
model4 = Sequential()
model4.add(Embedding(vocab_size, 32, input_length=max_len))
model4.add(Dropout(0.3))
model4.add(Conv1D(32, 5, activation='relu'))
model4.add(MaxPooling1D(pool_size=4))
model4.add(LSTM(32))
model4.add(Dense(21, activation='softmax'))
model4.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 32)           597056    
_________________________________________________________________
dropout (Dropout)            (None, 300, 32)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 296, 32)           5152      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 74, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 21)                693       
Total params: 611,221
Trainable params: 611,221
Non-trainable params: 0
__________________________________________________

In [ ]:
model4.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = .0005), metrics=['acc'])

In [ ]:
callback_ear = [EarlyStopping(monitor = 'val_acc', mode = 'max', patience = 2, verbose = 1), ModelCheckpoint(filepath = os.path.join(path, '1028_4_big1.h5'), monitor = 'val_acc', save_best_only = True, mode = 'max', verbose = 1)]

history4 = model4.fit(pad_X_train, del_y_train, epochs=4, batch_size=64, validation_split=val_rate, verbose = 1,shuffle = True, callbacks= callback_ear)

Epoch 1/4
14245/14245 [==============================] - 380s 25ms/step - loss: 1.2642 - acc: 0.6264 - val_loss: 0.7507 - val_acc: 0.7914

Epoch 00001: val_acc improved from -inf to 0.79136, saving model to /content/drive/MyDrive/Colab Notebooks/1028_4_big1.h5
Epoch 2/4
14245/14245 [==============================] - 346s 24ms/step - loss: 0.7449 - acc: 0.7915 - val_loss: 0.6749 - val_acc: 0.8096

Epoch 00002: val_acc improved from 0.79136 to 0.80959, saving model to /content/drive/MyDrive/Colab Notebooks/1028_4_big1.h5
Epoch 3/4
14245/14245 [==============================] - 345s 24ms/step - loss: 0.6878 - acc: 0.8050 - val_loss: 0.6456 - val_acc: 0.8151

Epoch 00003: val_acc improved from 0.80959 to 0.81505, saving model to /content/drive/MyDrive/Colab Notebooks/1028_4_big1.h5
Epoch 4/4
14245/14245 [==============================] - 345s 24ms/step - loss: 0.6627 - acc: 0.8097 - val_loss: 0.6296 - val_acc: 0.8175

Epoch 00004: val_acc improved from 0.81505 to 0.81746, saving model to /

In [ ]:
model4 = load_model( os.path.join(path, '1028_4_big1.h5'))  #0.69781

In [ ]:
pred_val = model4.predict(pad_X_val)

In [ ]:
import numpy as np


score_board = []
for true,pred in zip(bigval_y,pred_val):
  if np.argmax(true) == np.argmax(pred):
    score_board.append(1)
  else:
    score_board.append(0)
print('Accuracy : ',np.array(score_board).sum()/len(score_board) * 100)

Accuracy :  81.70033539044935


In [ ]:
pred_val

array([[3.8335096e-05, 2.6070560e-05, 1.0403705e-02, ..., 2.4596415e-04,
        1.4551587e-12, 1.1218764e-11],
       [2.8404456e-03, 3.5714819e-03, 6.9607586e-01, ..., 2.1442219e-03,
        2.5906348e-11, 1.5063077e-10],
       [2.2976402e-04, 8.3594859e-06, 1.9571345e-01, ..., 1.2317990e-03,
        3.7066289e-13, 1.5737728e-12],
       ...,
       [2.7306031e-03, 2.3187125e-05, 5.3173292e-02, ..., 1.9049007e-02,
        6.2815469e-14, 6.5906341e-13],
       [1.7239050e-03, 2.9654280e-05, 8.8866997e-01, ..., 1.9291746e-03,
        2.3624364e-10, 4.1914636e-10],
       [1.6391819e-05, 5.8631034e-05, 3.0829821e-04, ..., 1.1958946e-03,
        2.3257720e-12, 4.8207705e-12]], dtype=float32)

In [ ]:
big = sorted(data['Big'].unique())

In [ ]:
val = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_val.csv')

In [ ]:
pred_big_val = []
for i in pred_val:
  pred_idx = np.argmax(i)
  pred_big_val.append(big[pred_idx])

val['big4'] = pred_big_val

In [ ]:
val.to_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_val.csv',index=False)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_test.csv')

In [ ]:
test.shape

(19159, 2)

In [ ]:
pred_X_test = model4.predict(pad_X_test)

In [ ]:
pred_big_test = []
for i in pred_X_test:
  pred_idx = np.argmax(i)
  pred_big_test.append(big[pred_idx])

In [ ]:
test['big4']=pred_big_test
test

,big2,big3,big4
0,F,F,F
1,C,C,C
2,F,F,F
3,L,L,L
4,A,A,A
...,...,...,...
19154,M,M,M
19155,C,C,C
19156,A,A,A
19157,J,J,J


In [ ]:
test.to_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_test.csv',index=False)

In [ ]:
pred_train = model4.predict(pad_X_train)

In [ ]:
pred_big_train = []
for i in pred_train:
  pred_idx = np.argmax(i)
  pred_big_train.append(big[pred_idx])

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_train4.pickle', 'wb') as f:
    pickle.dump(pred_big_train, f)

In [ ]:
#with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/ysb_predict_data/m4_pred.pickle', 'wb') as f:
   # pickle.dump(pred_val, f)